In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec("/content/drive/MyDrive/try/glove.6B.300d.txt", "glove_model2.txt")

(400000, 300)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("merged_data_2.csv", sep=";")
df

,Column1,Course code,Course Name,Long descrition,Description,Prerequisites:,Semester,Format,CRN,Subject,Course,Type,Credits,Instructor,Status
0,0,ANAT 214,Systemic Human Anatomy.,Introduction to the gross anatomy of the vario...,Introduction to the gross anatomy of the vario...,"BIOL 112 (or CEGEP equivalent), PHGY 209 and P...",Fall,NaN,1181,ANAT,214,Lecture,30,"Mette Calve Coleman, Gabriel Venne",Active
1,1,ANAT 261,Introduction to Dynamic Histology.,An introduction to light and electron microsco...,An introduction to light and electron microsco...,BIOL 112 or CEGEP equivalent,Fall,NaN,1184,ANAT,261,Lecture,40,"Craig A. Mandato, Carlos R Morales",Active
2,2,ANAT 315,Clinical Human Musculoskeletal Anatomy.,The regional human gross anatomy of the skelet...,The regional human gross anatomy of the skelet...,NaN,Fall,NaN,1188,ANAT,315,Lecture,30,Campbell Rolian,Active
3,3,ANAT 321,Circuitry of the Human Brain.,This course explores the functional organizati...,This course explores the functional organizati...,ANAT/BIOC 212 or BIOL 201; and one of PHGY 209...,Fall,NaN,1191,ANAT,321,Lecture,30,David S Ragsdale,Active
4,4,ANAT 323,Clinical Neuroanatomy.,This course will focus on the structural and f...,This course will focus on the structural and f...,ANAT 315 and ANAT 316 or ANAT 214 and ANAT 314.,NaN,NaN,1192,ANAT,323,Lecture,30,Mikaela Liis Stiver,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,357,PSYC 535,Advanced Topics in Social Psychology.,Classic and contemporary readings in a specifi...,Classic and contemporary readings in a specifi...,PSYC 215; and PSYC 333 or PSYC 351 or PSYC 380.,Winter,NaN,5147,PSYC,535,Seminar,30,Jordan Robert Axt,Active
316,358,PSYC 538,"Categorization, Communication and Consciousness.",The main challenges that cognitive science fac...,The main challenges that cognitive science fac...,Student must have completed a 300 level course...,NaN,NaN,8308,PSYC,538,Seminar,30,Stevan Harnad,Active
317,359,PSYC 545,Topics in Language Acquisition.,Psychological mechanisms and theories of first...,Psychological mechanisms and theories of first...,NaN,Fall,NaN,7840,PSYC,545,Seminar,30,Kristine Onishi,Active
318,360,PSYT 199,FYS: Mental Illness and the Brain.,This course will introduce the student to the ...,This course will introduce the student to the ...,NaN,NaN,NaN,5164,PSYT,199,Seminar,30,Reut Gruber,Active


# Data cleaning

Loop over the long descriptions and remove stop words + lemmatise the other words

In [ ]:
# Setup
!pip install -q wordcloud
!pip install -U scikit-learn
!pip install --user -U nltk
!pip install pandas
!pip install wn

import nltk
import csv
import pandas as pd
import numpy as np
import re
import wn

from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('nltk.tokenize')
nltk.download('omw-1.4')
from nltk.corpus.reader.tagged import word_tokenize


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 75 kB 2.4 MB/s 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Error loading nltk.tokenize: Package 'nltk.tokenize' not
[nltk_data]     found in index
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
#upload file
df = pd.read_csv("/content/merged_data_2.csv", sep=";")

#only select the descriptions
descriptions = df["Description"]


def clean_up(l):
  
  stopwords = nltk.corpus.stopwords.words('english')
  stemmer = nltk.stem.PorterStemmer()
  lemmatizer = nltk.stem.WordNetLemmatizer()
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  

  l = re.sub(r'\d+', '', str(l)) #Remove numbers from the descriptions
  l = re.sub("/(\r\n|\n|\r)/gm", "", str(l)) #Remove \n from the descriptions
  l_tok = tokenizer.tokenize(l) #Remove all punctuation from the descriptions
  l1 = [w.lower() for w in l_tok if not w.lower() in stopwords]  #Remove all the stopwords and lowercase words
  l_filtered = [lemmatizer.lemmatize(word).lower() for word in l1]
  
  line = " ".join(l_filtered)

  return line

df["clean_description"] = df.apply(lambda row: clean_up(row["Description"]), axis=1)




In [ ]:

#only select the long description
description_long = df['Long descrition']

def get_list_courses(line):
  stopwords = nltk.corpus.stopwords.words('english')
  stemmer = nltk.stem.PorterStemmer()
  lemmatizer = nltk.stem.WordNetLemmatizer()
  
  #get line that contains Restriction or Restrictions until a period
  #get substring starting with a class name until a period, only 1 restriction sentence => NOPE HAHAHA (until number =)

  try:
    unclean = re.search(r".*Restriction.*:\s*([^\n\r]*)*\s*([^\n\r]*)*\s*([^0-9]*)", line) 
    almost_clean = []
    for n in range(100):
      try:
        almost_clean += re.search(r"([A-Z]{4}).*( )?.*([0-9]{3}).*(.)", unclean.group(n)) 
        return almost_clean
      except:
        return re.search(r"([A-Z]{4}).*( )?.*([0-9]{3}).*(.)", unclean.group(n)).group(0) 
  except AttributeError: #if no course code clear restriction -> possible improvement
    return 'nan'

#to be cleaned manually still
df["Clean restriction"] = df.apply(lambda row: get_list_courses(row["Long descrition"]), axis=1)


#New CSV containing course code and to-be-cleaned restrictions
f = pd.DataFrame()
f["Course code"] = df["Course code"] #copy onto new csv
f["Clean restriction"] = df["Clean restriction"]


In [ ]:
df.head()

,Column1,Course code,Course Name,Long descrition,Description,Prerequisites:,Semester,Format,CRN,Subject,Course,Type,Credits,Instructor,Status,clean_description,Clean restriction
0,0,ANAT 214,Systemic Human Anatomy.,Introduction to the gross anatomy of the vario...,Introduction to the gross anatomy of the vario...,"BIOL 112 (or CEGEP equivalent), PHGY 209 and P...",Fall,NaN,1181,ANAT,214,Lecture,30,"Mette Calve Coleman, Gabriel Venne",Active,introduction gross anatomy various organ syste...,nan
1,1,ANAT 261,Introduction to Dynamic Histology.,An introduction to light and electron microsco...,An introduction to light and electron microsco...,BIOL 112 or CEGEP equivalent,Fall,NaN,1184,ANAT,261,Lecture,40,"Craig A. Mandato, Carlos R Morales",Active,introduction light electron microscopic anatom...,nan
2,2,ANAT 315,Clinical Human Musculoskeletal Anatomy.,The regional human gross anatomy of the skelet...,The regional human gross anatomy of the skelet...,NaN,Fall,NaN,1188,ANAT,315,Lecture,30,Campbell Rolian,Active,regional human gross anatomy skeleton joint mu...,ANAT 314.
3,3,ANAT 321,Circuitry of the Human Brain.,This course explores the functional organizati...,This course explores the functional organizati...,ANAT/BIOC 212 or BIOL 201; and one of PHGY 209...,Fall,NaN,1191,ANAT,321,Lecture,30,David S Ragsdale,Active,course explores functional organization human ...,nan
4,4,ANAT 323,Clinical Neuroanatomy.,This course will focus on the structural and f...,This course will focus on the structural and f...,ANAT 315 and ANAT 316 or ANAT 214 and ANAT 314.,NaN,NaN,1192,ANAT,323,Lecture,30,Mikaela Liis Stiver,Active,course focus structural functional relationshi...,ANAT 321.


# Nouvelle section

# Extracting Rate my professor reviews

In [ ]:
#!pip install requests

Create a class that will extract professors' Ratemyprof information, and create an object of that class using the McGill Ratemyprof ID 1439

In [ ]:
import requests
import json
import math

class RateMyProfScraper:
        def __init__(self,schoolid):
            self.UniversityId = schoolid
            self.professorlist = self.createprofessorlist()
            self.indexnumber = False

        def createprofessorlist(self):#creates List object that include basic information on all Professors from the IDed University
            tempprofessorlist = []
            num_of_prof = self.GetNumOfProfessors(self.UniversityId)
            num_of_pages = math.ceil(num_of_prof / 20)
            i = 1
            while (i <= num_of_pages):# the loop insert all professor into list
                page = requests.get("http://www.ratemyprofessors.com/filter/professor/?&page=" + str(
                    i) + "&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid=" + str(
                    self.UniversityId))
                temp_jsonpage = json.loads(page.content)
                temp_list = temp_jsonpage['professors']
                tempprofessorlist.extend(temp_list)
                i += 1
            return tempprofessorlist

        def GetNumOfProfessors(self,id):  # function returns the number of professors in the university of the given ID.
            page = requests.get(
                "http://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid=" + str(
                    id))  # get request for page
            temp_jsonpage = json.loads(page.content)
            num_of_prof = temp_jsonpage[
                              'remaining'] + 20  # get the number of professors at William Paterson University
            return num_of_prof

        def SearchProfessor(self, ProfessorName):
            self.indexnumber = self.GetProfessorIndex(ProfessorName)
            return self.PrintProfessorInfo()
            #return self.indexnumber

        def GetProfessorIndex(self,ProfessorName):  # function searches for professor in list
            names=ProfessorName.split(" ")
            for i in range(0, len(self.professorlist)):
                if (self.professorlist[i]['tFname'] == names[0]) and (self.professorlist[i]['tLname']==names[-1]):
                    return i
            return False  # Return False is not found

        def PrintProfessorInfo(self):  # print search professor's name and RMP score
            if self.indexnumber == False:
                return False
            else:
                return self.professorlist[self.indexnumber]

        def PrintProfessorDetail(self,key):  # print search professor's name and RMP score
            if self.indexnumber == False:
                print("error")
                return "error"
            else:
                print(self.professorlist[self.indexnumber][key])
                return self.professorlist[self.indexnumber][key]

McGillUni=RateMyProfScraper(1439)

In [ ]:
df.head()

,Column1,Course code,Course Name,Long descrition,Description,Prerequisites:,Semester,Format,CRN,Subject,Course,Type,Credits,Instructor,Status,clean_description,Clean restriction
0,0,ANAT 214,Systemic Human Anatomy.,Introduction to the gross anatomy of the vario...,Introduction to the gross anatomy of the vario...,"BIOL 112 (or CEGEP equivalent), PHGY 209 and P...",Fall,NaN,1181,ANAT,214,Lecture,30,"Mette Calve Coleman, Gabriel Venne",Active,introduction gross anatomy various organ syste...,nan
1,1,ANAT 261,Introduction to Dynamic Histology.,An introduction to light and electron microsco...,An introduction to light and electron microsco...,BIOL 112 or CEGEP equivalent,Fall,NaN,1184,ANAT,261,Lecture,40,"Craig A. Mandato, Carlos R Morales",Active,introduction light electron microscopic anatom...,nan
2,2,ANAT 315,Clinical Human Musculoskeletal Anatomy.,The regional human gross anatomy of the skelet...,The regional human gross anatomy of the skelet...,NaN,Fall,NaN,1188,ANAT,315,Lecture,30,Campbell Rolian,Active,regional human gross anatomy skeleton joint mu...,ANAT 314.
3,3,ANAT 321,Circuitry of the Human Brain.,This course explores the functional organizati...,This course explores the functional organizati...,ANAT/BIOC 212 or BIOL 201; and one of PHGY 209...,Fall,NaN,1191,ANAT,321,Lecture,30,David S Ragsdale,Active,course explores functional organization human ...,nan
4,4,ANAT 323,Clinical Neuroanatomy.,This course will focus on the structural and f...,This course will focus on the structural and f...,ANAT 315 and ANAT 316 or ANAT 214 and ANAT 314.,NaN,NaN,1192,ANAT,323,Lecture,30,Mikaela Liis Stiver,Active,course focus structural functional relationshi...,ANAT 321.


Adding the reviews to the csv

In [ ]:
df["Instructor"]= df["Instructor"].str.split(", ", n = -1, expand = False)
i=0
df["Average rating"]=""
while i<320:
    reviews=0
    profs= 0
    for instructor in df["Instructor"][i]:
        profpage=McGillUni.SearchProfessor(instructor)
        if profpage!=False:
            rating = profpage.get('overall_rating')
            if rating != 'N/A':
                reviews+=float(rating)
                profs+=1
    if profs!=0:
        df["Average rating"][i]=reviews/profs
    else:
        df["Average rating"][i]=-1
    
    i+=1

df.to_csv("ratings.csv", sep=";")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# NLP

In [ ]:
from array import array
import pandas as pd
import numpy as np




dict_from_csv = pd.read_csv("/content/first_prerequeisite.csv", sep=";" , header=None, index_col=0, squeeze=True).to_dict()
dict_from_csv = {key: list(map(str, str(value).split(","))) for key, value in dict_from_csv.items()}


# creating an empty list
lst = []
done = 'done'

while True:
    val = input("what classes did you take ? (enter at each class and write done at the end) ")
    if val == done:
        break
    lst.append(val)


can_take = []
for course in dict_from_csv:
    out = False
    for pre_req in dict_from_csv[course]:
        if pre_req == "nan" :
            out == False
        elif 'or' in pre_req:
            pre_reqs_or = pre_req.split(" or ")
            for p in pre_reqs_or:
                if p in lst:
                    out = False
                    break
                else:
                    out = True  
        elif  pre_req not in lst:
            out = True
            break


    
    if out == False:
        can_take.append(course)

print(can_take)

what classes did you take ? (enter at each class and write done at the end) done
['ANAT 315 ', 'ATOC 181 ', 'ATOC 182 ', 'ATOC 185 ', 'BIOL 111 ', 'BIOL 115 ', 'BIOL 210 ', 'BIOL 499D1 ', 'BIOL 507 ', 'BIOT 505 ', 'CHEM 110 ', 'CHEM 183 ', 'COMP 202 ', 'COMP 204 ', 'COMP 250 ', 'COMP 402D1 ', 'COMP 480 ', 'COMP 546 ', 'COMP 557 ', 'COMP 596 ', 'COMP 597 ', 'COMP 598 ', 'COMP 599 ', 'ENVR 200 ', 'ENVR 201 ', 'ENVR 203 ', 'ENVR 301 ', 'ENVR 400 ', 'ENVR 485 ', 'ENVR 490 ', 'ENVR 491 ', 'EPSC 185 ', 'EPSC 201 ', 'EPSC 220 ', 'EPSC 233 ', 'EPSC 240 ', 'EPSC 478 ', 'EPSC 550 ', 'EPSC 551 ', 'EPSC 552 ', 'EPSC 570 ', 'ESYS 500 ', 'EXMD 502 ', 'EXMD 504 ', 'EXMD 509 ', 'EXMD 510 ', 'FSCI 198 ', 'GEOG 200 ', 'GEOG 201 ', 'GEOG 202 ', 'GEOG 203 ', 'GEOG 216 ', 'GEOG 290 ', 'GEOG 302 ', 'GEOG 308 ', 'GEOG 398 ', 'GEOG 407 ', 'GEOG 408 ', 'GEOG 470 ', 'GEOG 489 ', 'GEOG 492D1 ', 'MATH 122 ', 'MATH 123 ', 'MATH 133 ', 'MATH 139 ', 'MATH 140 ', 'MATH 150 ', 'MATH 180 ', 'MATH 203 ', 'MATH 228 ', 'M

# Output

In [ ]:
df

,Column1,Course code,Course Name,Long descrition,Description,Prerequisites:,Semester,Format,CRN,Subject,Course,Type,Credits,Instructor,Status,clean_description,Clean restriction,Average rating
0,0,ANAT 214,Systemic Human Anatomy.,Introduction to the gross anatomy of the vario...,Introduction to the gross anatomy of the vario...,"BIOL 112 (or CEGEP equivalent), PHGY 209 and P...",Fall,NaN,1181,ANAT,214,Lecture,30,"[Mette Calve Coleman, Gabriel Venne]",Active,introduction gross anatomy various organ syste...,nan,4.8
1,1,ANAT 261,Introduction to Dynamic Histology.,An introduction to light and electron microsco...,An introduction to light and electron microsco...,BIOL 112 or CEGEP equivalent,Fall,NaN,1184,ANAT,261,Lecture,40,"[Craig A. Mandato, Carlos R Morales]",Active,introduction light electron microscopic anatom...,nan,3.75
2,2,ANAT 315,Clinical Human Musculoskeletal Anatomy.,The regional human gross anatomy of the skelet...,The regional human gross anatomy of the skelet...,NaN,Fall,NaN,1188,ANAT,315,Lecture,30,[Campbell Rolian],Active,regional human gross anatomy skeleton joint mu...,ANAT 314.,-1
3,3,ANAT 321,Circuitry of the Human Brain.,This course explores the functional organizati...,This course explores the functional organizati...,ANAT/BIOC 212 or BIOL 201; and one of PHGY 209...,Fall,NaN,1191,ANAT,321,Lecture,30,[David S Ragsdale],Active,course explores functional organization human ...,nan,-1
4,4,ANAT 323,Clinical Neuroanatomy.,This course will focus on the structural and f...,This course will focus on the structural and f...,ANAT 315 and ANAT 316 or ANAT 214 and ANAT 314.,NaN,NaN,1192,ANAT,323,Lecture,30,[Mikaela Liis Stiver],Active,course focus structural functional relationshi...,ANAT 321.,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,357,PSYC 535,Advanced Topics in Social Psychology.,Classic and contemporary readings in a specifi...,Classic and contemporary readings in a specifi...,PSYC 215; and PSYC 333 or PSYC 351 or PSYC 380.,Winter,NaN,5147,PSYC,535,Seminar,30,[Jordan Robert Axt],Active,classic contemporary reading specific content ...,nan,4.0
316,358,PSYC 538,"Categorization, Communication and Consciousness.",The main challenges that cognitive science fac...,The main challenges that cognitive science fac...,Student must have completed a 300 level course...,NaN,NaN,8308,PSYC,538,Seminar,30,[Stevan Harnad],Active,main challenge cognitive science face today fo...,nan,-1
317,359,PSYC 545,Topics in Language Acquisition.,Psychological mechanisms and theories of first...,Psychological mechanisms and theories of first...,NaN,Fall,NaN,7840,PSYC,545,Seminar,30,[Kristine Onishi],Active,psychological mechanism theory first language ...,nan,-1
318,360,PSYT 199,FYS: Mental Illness and the Brain.,This course will introduce the student to the ...,This course will introduce the student to the ...,NaN,NaN,NaN,5164,PSYT,199,Seminar,30,[Reut Gruber],Active,course introduce student fundamental neuroscie...,nan,4.0


In [ ]:
new_df = pd.DataFrame(columns=['Course code', 'clean_description','Description'])
for index, row in df.iterrows():
  for c in can_take:
    # if row['Course code'] == "PSYC 498D2 ":
      # print(row['Course code'],c)
      # print(len(row['Course code'] ) , len(c))
    if row['Course code'] == c:
      
      new_df = new_df.append({'Course code': row['Course code'], 'clean_description': row['clean_description'], 'Description': row['Description']},ignore_index = True)

  # dfObj = dfObj.append({'User_ID': 23, 'UserName': 'Riti', 'Action': 'Login'}, ignore_index=True)

In [ ]:
new_df

,Course code,clean_description,Description
0,ANAT 315,regional human gross anatomy skeleton joint mu...,The regional human gross anatomy of the skelet...
1,ATOC 181,survey earth atmosphere weather climate system...,"A survey of the Earth's atmosphere, weather an..."
2,ATOC 182,air sea interaction oceanic property global cl...,Air-sea interaction; oceanic properties; globa...
3,ATOC 185,course examines science behind different type ...,This course examines the science behind differ...
4,BIOL 111,introduction phylogeny structure function adap...,"An introduction to the phylogeny, structure, f..."
...,...,...,...
106,PSYC 498D2,nan,NaN
107,PSYC 526,examine detail structure visual system functio...,We examine in detail the structure of the visu...
108,PSYC 538,main challenge cognitive science face today fo...,The main challenges that cognitive science fac...
109,PSYC 545,psychological mechanism theory first language ...,Psychological mechanisms and theories of first...


In [ ]:
new_df.dropna(inplace = True)

In [ ]:
new_df

,Course code,clean_description,Description
0,ANAT 315,regional human gross anatomy skeleton joint mu...,The regional human gross anatomy of the skelet...
1,ATOC 181,survey earth atmosphere weather climate system...,"A survey of the Earth's atmosphere, weather an..."
2,ATOC 182,air sea interaction oceanic property global cl...,Air-sea interaction; oceanic properties; globa...
3,ATOC 185,course examines science behind different type ...,This course examines the science behind differ...
4,BIOL 111,introduction phylogeny structure function adap...,"An introduction to the phylogeny, structure, f..."
...,...,...,...
104,PSYC 482,ethical issue scientific clinical psychology s...,Ethical issues in scientific and clinical psyc...
107,PSYC 526,examine detail structure visual system functio...,We examine in detail the structure of the visu...
108,PSYC 538,main challenge cognitive science face today fo...,The main challenges that cognitive science fac...
109,PSYC 545,psychological mechanism theory first language ...,Psychological mechanisms and theories of first...


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.feature_extraction.text import TfidVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer



In [ ]:
def reccomend(course_name, course_taken, df = df):
  dict_from_csv = pd.read_csv("/content/first_prerequeisite.csv", sep=";" , header=None, index_col=0, squeeze=True).to_dict()
  dict_from_csv = {key: list(map(str, str(value).split(","))) for key, value in dict_from_csv.items()}


  # creating an empty list
  lst = []
  lst = course_taken

  can_take = []
  for course in dict_from_csv:
      out = False
      for pre_req in dict_from_csv[course]:
          if pre_req == "nan" :
              out == False
              break
          elif 'or' in pre_req:
              pre_reqs_or = pre_req.split(" or ")
              for p in pre_reqs_or:
                  if p in lst:
                      out = False
                      break
                  else:
                      out = True  
          elif  pre_req not in lst:
              out = True
              break
      if out == False:
          can_take.append(course)


  can_take = df['Course code'].to_list()
  new_df = pd.DataFrame(columns=['Course code', 'clean_description','Description'])
  # print(can_take)
  for index, row in df.iterrows():
    for c in can_take:
      if row['Course code'] == c:
        
        new_df = new_df.append({'Course code': row['Course code'], 'clean_description': row['clean_description'], 'Description': row['Description']},ignore_index = True)
  tf = TfidfVectorizer (analyzer='word', ngram_range=(1, 3), min_df= 0, stop_words='english')
  new_df[new_df.index.duplicated()]
  new_df['clean_description'] = new_df['clean_description'].dropna()

  tfidf_matrix = tf.fit_transform(new_df['clean_description'].values.astype('U'))
  cos_sim = cosine_similarity(tfidf_matrix,tfidf_matrix)
  # description = new_df["clean_description"].to_list()
  # print(new_df['Course code'].to_list())
  # try:
  print("course name is :" , course_name)
  index = new_df['Course code'].to_list().index(course_name + " ")
  description = new_df.at[index, 'clean_description']
  print(description)
  
  rec = []
  # idx = description[description == description].index[0]
  score_series = pd.Series(cos_sim[index]).sort_values(ascending = False)
  top_10_indexes = list(score_series.iloc[1:11].index)
  for i in top_10_indexes:
    # print(df.iloc[i,"Course Name"])
    rec.append( [df.at[i,'Course code'], df.at[i,'Course Name']])
  return rec
  # except:
    # print("Class not offered for the Fall semester or you do not have the pre-requisites")

In [ ]:
reccomend("COMP 251", [ 'COMP 206', 'COMP 250', 'COMP 202', 'MATH 324'],df)

course name is : COMP 251
introduction algorithm design analysis graph algorithm greedy algorithm data structure dynamic programming maximum flow


[['COMP 360 ', 'Algorithm Design.'],
 ['GEOG 333 ', 'Introduction to Programming for Spatial Sciences.'],
 ['COMP 204 ', 'Computer Programming for Life Sciences.'],
 ['COMP 451 ', 'Fundamentals of Machine Learning.'],
 ['COMP 250 ', 'Introduction to Computer Science.'],
 ['COMP 551 ', 'Applied Machine Learning.'],
 ['PSYC 305 ', 'Statistics for Experimental Design.'],
 ['COMP 302 ', 'Programming Languages and Paradigms.'],
 ['ATOC 512 ', 'Atmospheric and Oceanic Dynamics.'],
 ['MATH 417 ', 'Linear Optimization.']]

In [ ]:
course_name = 'COMP 251'
df['Course code'].to_list().index(course_name + " ")

77

In [ ]:
df.iloc[91]

Column1                                                             98
Course code                                                  COMP 424 
Course Name                                   Artificial Intelligence.
Long descrition      Introduction to search methods. Knowledge repr...
Description          Introduction to search methods. Knowledge repr...
Prerequisites:       (COMP 206 or ECSE 321), MATH 323 or equivalent...
Semester                                                           NaN
Format                                                             NaN
CRN                                                               2075
Subject                                                           COMP
Course                                                             424
Type                                                           Lecture
Credits                                                             30
Instructor                                          [David Paul Meger]
Status

In [ ]:
sents = [ i.split() for i in df.clean_description.to_list()]

In [ ]:
# from gensim.models import Word2Vec
# model = Word2Vec(sents,min_count=1,size=32)
# # model = Word2Vec(size=300, min_count=1)
# # model.build_vocab(sents)
# total_examples = model.corpus_count

# # Save the vocab of your dataset
# vocab = list(model.wv.vocab.keys())

from gensim.models import Word2Vec

model = Word2Vec(size=300, min_count=1)
model.build_vocab(sents)
total_examples = model.corpus_count

# Save the vocab of your dataset
vocab = list(model.wv.vocab.keys())

In [ ]:
from gensim.models import KeyedVectors

pretrained_path = "/content/glove_model2.txt"
model_2 = KeyedVectors.load_word2vec_format(pretrained_path, binary=False)

# Add the pre-trained model vocabulary
model.build_vocab([list(model_2.vocab.keys())], update=True)

# Load the pre-trained models embeddings
# note: if a word doesn't exist in the pre-trained vocabulary then it is left as is in the original model
model.intersect_word2vec_format(pretrained_path, binary=False, lockf=1.0)

In [ ]:
model.train(sents, total_examples=len(sents), epochs=model.epochs)

(36950, 36950)

In [ ]:
model.wv.get_vector('computer')

array([-2.88117051e-01,  1.49157107e-01,  1.10488117e-01, -6.59659982e-01,
        3.11957132e-02,  9.75730494e-02, -1.94530457e-01, -3.83433193e-01,
        6.09358490e-01, -2.18522573e+00,  2.36630514e-01, -2.54048891e-02,
        7.24419832e-01, -4.39331859e-01,  5.07281482e-01, -8.31133220e-03,
       -9.98720750e-02, -3.10123026e-01, -1.64748073e-01, -6.90751374e-02,
        2.02345654e-01,  2.98587173e-01, -9.73286256e-02, -1.81143090e-01,
       -5.83807766e-01,  3.59807521e-01,  4.23371315e-01,  1.82000592e-01,
        2.46590197e-01, -2.25706771e-01, -9.21682939e-02,  1.56204522e-01,
        1.89165756e-01,  3.29249978e-01, -8.06368887e-01, -2.01648310e-01,
       -2.34939694e-01, -1.04503324e-02, -2.09318072e-01, -1.77585091e-02,
        1.86248317e-01,  3.40122432e-01, -3.39063674e-01,  8.60076964e-01,
       -5.91472611e-02, -1.69150814e-01,  2.99067795e-01,  1.10971764e-01,
        3.22632343e-01, -8.13727379e-01,  9.08486307e-01, -3.96461815e-01,
        1.74564689e-01,  

In [ ]:
model.wv.n_similarity(['biology'], ['chemistry'])

0.71201694

In [ ]:
def reccomend(course_name, course_taken, df = df):

  dict_from_csv = pd.read_csv("/content/first_prerequeisite.csv", sep=";" , header=None, index_col=0, squeeze=True).to_dict()
  dict_from_csv = {key: list(map(str, str(value).split(","))) for key, value in dict_from_csv.items()}


  # creating an empty list
  # lst = []
  lst = course_taken

  can_take = []
  for course in dict_from_csv:
      out = False
      for pre_req in dict_from_csv[course]:
          if pre_req == "nan" :
              out == False
              break
          elif 'or' in pre_req:
              pre_reqs_or = pre_req.split(" or ")
              for p in pre_reqs_or:
                  if p in lst:
                      out = False
                      break
                  else:
                      out = True  
          elif pre_req.strip() not in lst:
              out = True
              break
      
      if out == False:
          can_take.append(course)

  
  # can_take = df['Course code'].to_list()
  new_df = pd.DataFrame(columns=['Course code', 'clean_description','Description','Course Name', 'Average rating'])
  # print(can_take)
  for index, row in df.iterrows():
    for c in can_take:
      if row['Course code'] == c:
        new_df = new_df.append({'Course code': row['Course code'], 'clean_description': row['clean_description'], 'Description': row['Description'], 'Course Name' : row['Course Name'], 'Average rating': row['Average rating']},ignore_index = True)
  tf = TfidfVectorizer (analyzer='word', ngram_range=(1, 3), min_df= 0, stop_words='english')
  # new_df[new_df.index.duplicated()]
  new_df['clean_description'] = new_df['clean_description'].dropna()

  try:
    index = new_df['Course code'].to_list().index(course_name + " ")
    print("Course input :" , course_name)
    des = new_df.at[index, 'clean_description']
    print("Course Name",  new_df.at[index, 'Course Name'])
    print("Description:", new_df.at[index, 'Description'])
    
    print("\n")
  # size(V)

    cos_sims = []
    for description in new_df['clean_description'].to_list():
      cos_sim = model.wv.n_similarity(des.split(), description.split())
      cos_sims.append(cos_sim)
    new_df['cos_sin'] = cos_sims
    # print(new_df.head())
    rec = []
    # idx = description[description == description].index[0]
    # score_series = pd.Series(cos_sim[index]).sort_values(ascending = False)
    new_df.sort_values(['cos_sin'],ascending=False,inplace = True)
    # top_10_indexes = list(score_series.iloc[1:11].index)\
    i = 0
    cs =[ str(i) + " " for i in course_taken]
    while len(rec) < 6:
      if new_df['Course code'].to_list()[i] not in cs:
        if new_df['Average rating'].to_list()[i] != -1:
          rec.append( [new_df['Course code'].to_list()[i], new_df['Course Name'].to_list()[i] , new_df['Average rating'].to_list()[i],new_df['Description'].to_list()[i]])
        elif new_df['Average rating'].to_list()[i] == -1:
          rec.append( [new_df['Course code'].to_list()[i], new_df['Course Name'].to_list()[i] , "NIL",new_df['Description'].to_list()[i]])
      i+=1
    for i in range(1,6):
      print("Class #" + str(i), ":" , rec[i][0], '\nCourse Name:' , rec[i][1], '\nDescription:',str(rec[i][3]).strip(),'\nAverage Professor rating:' ,str(rec[i][2]),'\n')
        # return rec[1:] 
    
  except:
    print("ERROR: Class not offered for the Fall semester or you do not have the pre-requisites")

In [ ]:
reccomend("COMP 551", ["COMP 251", 'COMP 206' , 'COMP 250', 'COMP 251', 'MATH 323', 'MATH 240', "MATH 223" , "MATH 222"],df)

Course input : COMP 551
Course Name Applied Machine Learning.
Description: Selected topics in machine learning and data mining, including clustering, neural networks, support vector machines, decision trees. Methods include feature selection and dimensionality reduction, error estimation and empirical validation, algorithm design and parallelization, and handling of large data sets. Emphasis on good methods and practices for deployment of real systems. 


Class #1 : COMP 424  
Course Name: Artificial Intelligence. 
Description: Introduction to search methods. Knowledge representation using logic and probability. Planning and decision making under uncertainty. Introduction to machine learning. 
Average Professor rating: 3.2 

Class #2 : PSYC 319  
Course Name: Computational Models - Cognition. 
Description: Introduction to the purpose, design, and role of computational modeling in the cognitive sciences. Domains examined will include memory, categorization, and language. Focus will be o

In [ ]:
reccomend("COMP 303", ["COMP 251", 'COMP 206' , 'COMP 250', 'COMP 251', 'MATH 323', 'COMP 324', 'MATH 240'],df)

Course input : COMP 303
Course Name Software Design.
Description: Principles, mechanisms, techniques, and tools for object-oriented software design and its implementation, including encapsulation, design patterns, and unit testing. 
 


Class #1 : COMP 361D1  
Course Name: Software Engineering Project. 
Description: Software development process in practice: requirement elicitation and analysis, software design, implementation, integration, test planning, and maintenance. Application of the core concepts and techniques through the realization of a large software system. 
Average Professor rating: NIL 

Class #2 : COMP 551  
Course Name: Applied Machine Learning. 
Description: Selected topics in machine learning and data mining, including clustering, neural networks, support vector machines, decision trees. Methods include feature selection and dimensionality reduction, error estimation and empirical validation, algorithm design and parallelization, and handling of large data sets. Empha

In [ ]:
reccomend("MATH 133", ["COMP 251", 'COMP 206' , 'COMP 250', 'COMP 251', 'MATH 323', 'COMP 324', 'MATH 240', "MATH 222", "MATH 133"],df)

Course input : MATH 133
Course Name Linear Algebra and Geometry.
Description: Systems of linear equations, matrices, inverses, determinants; geometric vectors in three dimensions, dot product, cross product, lines and planes; introduction to vector spaces, linear dependence and independence, bases. Linear transformations. Eigenvalues and diagonalization.


Class #1 : MATH 223  
Course Name: Linear Algebra. 
Description: Review of matrix algebra, determinants and systems of linear equations. Vector spaces, linear operators and their matrix representations, orthogonality. Eigenvalues and eigenvectors, diagonalization of Hermitian matrices. Applications. 
Average Professor rating: NIL 

Class #2 : MATH 248  
Course Name: Honours Vector Calculus. 
Description: Partial derivatives and differentiation of functions in several variables; Jacobians; maxima and minima; implicit functions. Scalar and vector fields; orthogonal curvilinear coordinates. Multiple integrals; arc length, volume and sur

In [ ]:
while True:
  continue